In [3]:
!pip -q install --upgrade transformers accelerate bitsandbytes einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 14.9 MB/s eta 0:00:00


In [4]:
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer

In [5]:
MODEL_ID = "Qwen/Qwen3-8B"

In [6]:
# Configuración de cuantización
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    use_fast=True,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    dtype=torch.float16,
    trust_remote_code=True
)
model.eval()

#                                  -------------------------------
#           ---------             | --------                  --- |            -----------
# texto -> |Tokenizer| - > IDs -> ||Codebook| -> Vectores -> |LLM|| -> IDs -> |unTokenizer| -> Texto (output)
#           ---------             | --------                  --- |            -----------
#                                  -------------------------------

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear4bit(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06

In [7]:
def qwen( prompt: str,
          system: str = "You are a logical planning assitant",
          max_new_tokens: int=512) -> str:

    # Alta temperatura: Creativo (alucinar)
    #                   Las probabilidades seran homogeneas
    # Menor temperatura: Más determinista
    #                    Se aproxima a un one-hot

    # enable_thinking: Habilita el modo de pensamiento
    # do_sample: Muestre aleatorios entre tokens más probables
    # top-p: Los token más probables hasta tener p de probabilidad
    # top-k: Seleccionamos los k tokens mas probables
    # stream: Controla si los tokens se van generando en tiempo real

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]

    # ID de tokens (respuesta)
    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
        enable_thinking=False
    )
    # Traducimos los Ids como texto
    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # Parámetros estrictos para tareas lógicas
    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=False,      # Apagar muestreo aleatorio
        temperature=0.01,     # Temperatura mínima
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )

    with torch.no_grad():
        # resp = [input, output]
        out = model.generate(**inputs, **gen_kwargs)

    # Sólo la parte nueva:
    gen_ids = out[0, inputs.input_ids.shape[1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True)

In [8]:
print(qwen(
    system="Eres un docente y respondes con precisión y concisión.",
    prompt="Cuentame que paso en el Jurasico.",
    max_new_tokens=500,
))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


El Jurásico fue una era geológica que ocurrió entre hace aproximadamente 201 y 145 millones de años. Durante este periodo:

- **La Tierra estaba dominada por grandes reptiles marinos y terrestres**, como los **plesiosaurios**, **ikthyosaurios** y **brontosaurios**.
- **Los dinosaurios eran los animales dominantes**, tanto en el mar como en la tierra.
- **La vegetación era abundante**, con árboles altos y plantas que proporcionaban alimento a los herbívoros.
- **El clima era cálido y húmedo**, con grandes extensiones de selvas tropicales.
- **Los mamíferos eran pequeños y nocturnos**, y no dominaban el ecosistema.
- **Hacia el final del Jurásico**, comenzaron a aparecer los primeros **aves** y **mamíferos más desarrollados**.

El Jurásico terminó con un gran evento de extinción masiva, que marcó el inicio del **Cretácico**.


In [9]:
def parse_logical_action(text_line: str) -> str:
    """
    Convierte el texto en lenguaje natural a la tupla lógica esperada.
    """
    text_line = text_line.strip().lower()

    # --- Dominio: Bloques ---
    if text_line.startswith("pick up"):
        block = re.search(r"pick up the (\w+) block", text_line).group(1)
        return f"(engage_payload {block})"
    elif text_line.startswith("put down"):
        block = re.search(r"put down the (\w+) block", text_line).group(1)
        return f"(release_payload {block})"
    elif text_line.startswith("unmount_node"):
        m = re.search(r"unmount_node the (\w+) block from on top of the (\w+) block", text_line)
        return f"(unmount_node {m.group(1)} {m.group(2)})"
    elif text_line.startswith("mount_node"):
        m = re.search(r"mount_node the (\w+) block on top of the (\w+) block", text_line)
        return f"(mount_node {m.group(1)} {m.group(2)})"

    # --- Dominio: Abstracto / Fantasía ---
    elif text_line.startswith("attack"):
        obj = re.search(r"attack object (\w+)", text_line).group(1)
        return f"(attack {obj})"
    elif text_line.startswith("succumb"):
        obj = re.search(r"succumb object (\w+)", text_line).group(1)
        return f"(succumb {obj})"
    elif text_line.startswith("feast"):
        m = re.search(r"feast object (\w+) from object (\w+)", text_line)
        return f"(feast {m.group(1)} {m.group(2)})"
    elif text_line.startswith("overcome"):
        m = re.search(r"overcome object (\w+) from object (\w+)", text_line)
        return f"(overcome {m.group(1)} {m.group(2)})"

    return None

In [10]:
with open("Task.json", "r") as f:
  tasks = json.load(f)

results = []

for i, task in enumerate(tasks):
  task_id = task["assembly_task_id"]
  scenario = task["scenario_context"]

  print(f"[{i+1}/{len(tasks)}] Procesando tarea: {task_id}")

  raw_response = qwen(prompt=scenario)

  print(raw_response, '\n')

[1/50] Procesando tarea: task_f6c3f52f55
Let's analyze your **plan** step by step, based on the **initial conditions** and the **rules** of the block system.

---

### ✅ **Initial Conditions:**
- The **yellow block** is **unobstructed**.
- The **orange block** is **unobstructed**.
- The **hand is empty**.
- The **red block** is on top of the **blue block**.
- The **orange block** is on top of the **red block**.
- The **blue block** is on the **table**.
- The **yellow block** is on the **table**.

### ✅ **Goal:**
- The **red block** is on top of the **orange block**.
- The **blue block** is on top of the **red block**.
- The **yellow block** is on top of the **blue block**.

---

### ✅ **Your Plan:**

1. **Unmount_node the red block from on top of the blue block**
   - ❌ **Invalid**.
     - The **red block** is **not on top of the blue block**.
     - The **red block** is **on top of the orange block**, not the blue block.
     - So, you **cannot unmount_node** the red block from the bl